## From EXR to YUV
* .exr images from ftp://svtopencontent.svt.se
* 1500 images at 50 fps = 30sec
* inside a .yuv raw video with 4k resolution and yuv420p sampling

import subprocess
command = f"ffmpeg -f concat -safe 0 -i tests_vids/srcRAW_FullHD/SVT/cuts_full.txt \
    -f yuv4mpegpipe tests_vids/srcRAW_FullHD/SVT/mux_fullmix_50.y4m"
subprocess.call(command, shell=True)

In [2]:
import subprocess
command = f"ffmpeg -f concat -safe 0 -i /home/ubuntu/video_dynopt/cuts.txt \
    -pix_fmt yuv420p -r 30 tests_vids/srcRAW_4K/bar_dinner.y4m"
subprocess.call(command, shell=True)

ffmpeg version N-106635-g83e1a1de88 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/home/ubuntu/ffmpeg_build --pkg-config-flags=--static --extra-cflags=-I/home/ubuntu/ffmpeg_build/include --extra-ldflags=-L/home/ubuntu/ffmpeg_build/lib --extra-libs='-lpthread -lm' --ld=g++ --bindir=/home/ubuntu/bin --enable-gpl --enable-gnutls --enable-libaom --enable-libass --enable-libfdk-aac --enable-libfreetype --enable-libmp3lame --enable-libopus --enable-libsvtav1 --enable-libdav1d --enable-libvorbis --enable-libvpx --enable-libx264 --enable-libx265 --enable-libvmaf --enable-version3 --enable-nonfree
  libavutil      57. 24.101 / 57. 24.101
  libavcodec     59. 26.100 / 59. 26.100
  libavformat    59. 22.100 / 59. 22.100
  libavdevice    59.  6.100 / 59.  6.100
  libavfilter     8. 33.100 /  8. 33.100
  libswscale      6.  6.100 /  6.  6.100
  libswresample   4.  6.100 /  4.  6.100
  libpostproc    56.  5.100 / 56.  5.100

0

import subprocess
c_seq = "ffmpeg -y -start_number 180000 -framerate 50 -i test_vids/src_waterfall/waterfall_%04d.exr \
        -c:v rawvideo -pix_fmt yuv420p -s 3840x2160 -r 50 test_vids/refs/waterfall.yuv"
subprocess.call(c_seq, shell=True)

In [1]:
import subprocess
c_seq = "ffmpeg -i /home/ubuntu/video_dynopt/tests_vids/srcRAW_4K/SVT/VIDEO_13cc0bda-5bf9-4fcc-8141-3af169a66e0a.mxf \
    -pix_fmt yuv420p -r 25 tests_vids/srcRAW_4K/SVT/edit_natural.y4m"
subprocess.call(c_seq, shell=True)

ffmpeg version N-106635-g83e1a1de88 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/home/ubuntu/ffmpeg_build --pkg-config-flags=--static --extra-cflags=-I/home/ubuntu/ffmpeg_build/include --extra-ldflags=-L/home/ubuntu/ffmpeg_build/lib --extra-libs='-lpthread -lm' --ld=g++ --bindir=/home/ubuntu/bin --enable-gpl --enable-gnutls --enable-libaom --enable-libass --enable-libfdk-aac --enable-libfreetype --enable-libmp3lame --enable-libopus --enable-libsvtav1 --enable-libdav1d --enable-libvorbis --enable-libvpx --enable-libx264 --enable-libx265 --enable-libvmaf --enable-version3 --enable-nonfree
  libavutil      57. 24.101 / 57. 24.101
  libavcodec     59. 26.100 / 59. 26.100
  libavformat    59. 22.100 / 59. 22.100
  libavdevice    59.  6.100 / 59.  6.100
  libavfilter     8. 33.100 /  8. 33.100
  libswscale      6.  6.100 /  6.  6.100
  libswresample   4.  6.100 /  4.  6.100
  libpostproc    56.  5.100 / 56.  5.100

0

## From YUV to YUV multires
the .yuv reference uncompressed file is downsized to 5 more resolutions

import numpy as np
import subprocess
source = "smithy"
res_widths = [3840, 2560, 1920, 1280, 1024, 640]
res_heights = [2160, 1440, 1080, 720, 576, 360]
for i in range(1, res_widths.size):
    path = "test_vids/downsampled/" + source + "_" + str(res_widths[i-1]) + ".yuv"
    out = "test_vids/downsampled/" + source + "_" + str(res_widths[i]) + ".yuv"
    command = f"ffmpeg -f rawvideo -video_size {res_widths[i-1]}x{res_heights[i-1]} -r 50 -pixel_format yuv420p \
    -i {path} -f rawvideo -c:v rawvideo -filter:v scale={res_widths[i]}x{res_heights[i]} -pix_fmt yuv420p {out}"
    subprocess.call(command, shell=True)

## From YUV to H264 AVC

In [29]:
import os
import numpy as np
import subprocess
def interval(l,n):
    w = (l[1] - l[0]) / (n - 1)
    return np.array([round(l[0]+i*w) for i in range(n)])
source = "/home/ubuntu/video_dynopt/tests_vids/srcRAW_SD/meridian_SD.y4m"
crfs = interval([15,45],8)
for current_crf in crfs:
    out = "tests_vids/testAVC/merid" + str(current_crf).zfill(2) + ".mp4"
    to_avc = f"ffmpeg -i {source} -c:v libx264 -crf {current_crf} -pix_fmt yuv420p {out} -hide_banner -loglevel error"
    subprocess.call(to_avc, shell=True)
    print(str(current_crf) + " done")

ffmpeg version N-106635-g83e1a1de88 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/home/ubuntu/ffmpeg_build --pkg-config-flags=--static --extra-cflags=-I/home/ubuntu/ffmpeg_build/include --extra-ldflags=-L/home/ubuntu/ffmpeg_build/lib --extra-libs='-lpthread -lm' --ld=g++ --bindir=/home/ubuntu/bin --enable-gpl --enable-gnutls --enable-libaom --enable-libass --enable-libfdk-aac --enable-libfreetype --enable-libmp3lame --enable-libopus --enable-libsvtav1 --enable-libdav1d --enable-libvorbis --enable-libvpx --enable-libx264 --enable-libx265 --enable-libvmaf --enable-version3 --enable-nonfree
  libavutil      57. 24.101 / 57. 24.101
  libavcodec     59. 26.100 / 59. 26.100
  libavformat    59. 22.100 / 59. 22.100
  libavdevice    59.  6.100 / 59.  6.100
  libavfilter     8. 33.100 /  8. 33.100
  libswscale      6.  6.100 /  6.  6.100
  libswresample   4.  6.100 /  4.  6.100
  libpostproc    56.  5.100 / 56.  5.100

45 done


frame=  300 fps=0.0 q=-1.0 Lsize=     667kB time=00:00:09.90 bitrate= 551.5kbits/s speed=23.2x    
video:663kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.624173%
[libx264 @ 0x56361a2e4180] frame I:4     Avg QP:29.06  size:  5433
[libx264 @ 0x56361a2e4180] frame P:106   Avg QP:29.94  size:  3076
[libx264 @ 0x56361a2e4180] frame B:190   Avg QP:32.11  size:  1739
[libx264 @ 0x56361a2e4180] consecutive B-frames:  9.7% 14.7%  9.0% 66.7%
[libx264 @ 0x56361a2e4180] mb I  I16..4:  1.3% 10.9% 87.9%
[libx264 @ 0x56361a2e4180] mb P  I16..4:  0.2%  1.0% 28.5%  P16..4: 17.2% 27.1% 24.5%  0.0%  0.0%    skip: 1.5%
[libx264 @ 0x56361a2e4180] mb B  I16..4:  0.2%  0.4%  8.3%  B16..8: 31.4% 26.4% 14.2%  direct:11.4%  skip: 7.7%  L0:45.1% L1:41.7% BI:13.3%
[libx264 @ 0x56361a2e4180] 8x8 transform intra:4.1% inter:9.7%
[libx264 @ 0x56361a2e4180] coded y,uvDC,uvAC intra: 95.5% 64.7% 29.7% inter: 48.4% 10.3% 1.0%
[libx264 @ 0x56361a2e4180] i16 v,h,dc,p: 39% 17% 23% 20%
[li

In [38]:
import os
import numpy as np
import subprocess
def interval(l,n):
    w = (l[1] - l[0]) / (n - 1)
    return np.array([round(l[0]+i*w) for i in range(n)])
source = "/home/ubuntu/video_dynopt/tests_vids/srcRAW_HD/in_to_tree_420_720p50.y4m"
out = "tests_vids/tree.mp4"
to_avc = f"ffmpeg -i {source} -c:v libx264 -crf 15 -pix_fmt yuv420p -vf 'fps=16,scale=320x180' {out} -hide_banner -loglevel error"
subprocess.call(to_avc, shell=True)
print(str(current_crf) + " done")

45 done


In [41]:
import os
import numpy as np
import subprocess
def interval(l,n):
    w = (l[1] - l[0]) / (n - 1)
    return np.array([round(l[0]+i*w) for i in range(n)])
source = "/home/ubuntu/video_dynopt/tests_vids/tree.mp4"
out = "tests_vids/treeBB.mp4"
to_avc = f"ffmpeg -flags2 +export_mvs -i {source} -vf codecview=mv=bb {out} -hide_banner -loglevel error"
subprocess.call(to_avc, shell=True)
print(str(current_crf) + " done")

45 done


## From YUV multires to H265 HEVC

In [21]:
import os
import numpy as np
import subprocess
def interval(l,n):
    w = (l[1] - l[0]) / (n - 1)
    return np.array([round(l[0]+i*w) for i in range(n)])
source = "tests_vids/srcRAW_HD/meridian_HD.y4m"
crfs = interval([12,27],8)
for current_crf in crfs:
    out = "tests_vids/testHEVC/medrid" + str(current_crf).zfill(2) + ".mp4"
    to_avc = f"ffmpeg -i {source} -c:v libx265 -crf {current_crf} -pix_fmt yuv420p {out} -hide_banner -loglevel error -x265-params log-level=none"
    subprocess.call(to_avc, shell=True)
    print(str(current_crf) + " done")

x265 [info]: HEVC encoder version 3.2.1+1-b5c86a64bbbe
x265 [info]: build info [Linux][GCC 9.3.0][64 bit] 8bit+10bit+12bit
x265 [info]: using cpu capabilities: MMX2 SSE2Fast LZCNT SSSE3 SSE4.2 AVX FMA3 BMI2 AVX2
x265 [info]: Main profile, Level-4 (Main tier)
x265 [info]: Thread pool created using 8 threads
x265 [info]: Slices                              : 1
x265 [info]: frame threads / pool features       : 3 / wpp(12 rows)
x265 [info]: Coding QT: max CU size, min CU size : 64 / 8
x265 [info]: Residual QT: max TU size, max depth : 32 / 1 inter / 1 intra
x265 [info]: ME / range / subpel / merge         : hex / 57 / 2 / 3
x265 [info]: Keyframe min / max / scenecut / bias: 25 / 250 / 40 / 5.00
x265 [info]: Lookahead / bframes / badapt        : 20 / 4 / 2
x265 [info]: b-pyramid / weightp / weightb       : 1 / 1 / 0
x265 [info]: References / ref-limit  cu / depth  : 3 / off / on
x265 [info]: AQ: mode / str / qg-size / cu-tree  : 2 / 1.0 / 32 / 1
x265 [info]: Rate Control / qCompress       

12 done


KeyboardInterrupt: 

## From RAW to VP9

In [17]:
import os
import numpy as np
import subprocess
def interval(l,n):
    w = (l[1] - l[0]) / (n - 1)
    return np.array([round(l[0]+i*w) for i in range(n)])
source = "tests_vids/srcRAW_4K/Netflix_BarScene_4096x2160_60fps_10bit_420.y4m"
crfs = interval([23,38],8)
for current_crf in crfs:
    out = "tests_vids/testVP9/bar" + str(current_crf).zfill(2) + ".mp4"
    to_avc = f"ffmpeg -i {source} -c:v libvpx-vp9 -crf {current_crf} -b:v 0 -pix_fmt yuv420p {out} -hide_banner -loglevel error"
    subprocess.call(to_avc, shell=True)
    print(str(current_crf) + " done")

39 done


# From RAW to AV1

In [19]:
import os
import numpy as np
import subprocess
def interval(l,n):
    w = (l[1] - l[0]) / (n - 1)
    return np.array([round(l[0]+i*w) for i in range(n)])
source = "tests_vids/srcRAW_4K/Netflix_BarScene_4096x2160_60fps_10bit_420.y4m"
out = "tests_vids/testAV1/bar" + str(37).zfill(2) + ".mp4"
to_avc = f"ffmpeg -i {source} -c:v libsvtav1 -crf {37} -pix_fmt yuv420p {out} -hide_banner -loglevel error"
subprocess.call(to_avc, shell=True)
print(str(37) + " done")

Svt[info]: -------------------------------------------
Svt[info]: SVT [version]:	SVT-AV1 Encoder Lib v1.0.0-rc2-7-g981ecdb0
Svt[info]: SVT [build]  :	GCC 9.4.0	 64 bit
Svt[info]: LIB Build date: Apr 21 2022 23:42:21
Svt[info]: -------------------------------------------
Svt[info]: Number of logical cores available: 8
Svt[info]: Number of PPCS 54
Svt[info]: [asm level on system : up to avx2]
Svt[info]: [asm level selected : up to avx2]
Svt[info]: -------------------------------------------
Svt[info]: SVT [config]: main profile	tier (auto)	level (auto)
Svt[info]: SVT [config]: width / height / fps numerator / fps denominator 		: 4096 / 2160 / 60 / 1
Svt[info]: SVT [config]: bit-depth / color format / compressed 10-bit format		: 8 / YUV420 / 0
Svt[info]: SVT [config]: preset / tune / pred struct 					: 10 / PSNR / random access
Svt[info]: SVT [config]: gop size / mini-gop size / key-frame type 			: 321 / 16 / key frame
Svt[info]: SVT [config]: BRC mode / rate factor 					: CRF / 37 
Svt[i

37 done
